# Tennis ball tracking using Kalman Filter

## Daniel Surpanu (4120700)

### Digital Signal and Image Processing

### Importing libraries

In [1]:
import numpy as np
import cv2 as cv
import imutils
import matplotlib.pyplot as plt
import time

### Helper functions

In [2]:
def show_frame(frame, frame_number):
	"""Function that visualizes the frame passed as parameter"""
	cv.imshow("Frame", frame)
	if cv.waitKey(0):
		cv.destroyAllWindows()

def run_frames(frames, one_by_one=False):
	"""Function that plays the frames passed as parameter in a video"""
	for i in range(len(frames)):
		if frames[i].size > 0:
			if one_by_one:
				show_frame(frames[i], i)
			else:
				cv.imshow("Frame", frames[i])
				cv.waitKey(1)
				
				time.sleep(0.05)
		else:
			break
	
	if cv.waitKey(0):
		cv.destroyAllWindows()

def set_initial_position(ball_positions):
	"""Function that controls if there is at leas a ball position,
		if not it sets the ball position to (0,0)
	"""
	if (len(ball_positions) > 0 and len(ball_positions[0])) > 0:
		return ball_positions[0][0][0], ball_positions[0][0][1]
	else:
		return 0, 0

def compute_nearest_measure(i, ball_positions, prev_measurement):
	"""Function that selects the nearest measure with respec to the previous one"""
	distances = []
	for j in range(len(ball_positions[i])):
		distance = ((prev_measurement[0] - ball_positions[i][j][0])**2 + (prev_measurement[1] - ball_positions[i][j][1])**2)
		distances.append([distance, ball_positions[i][j]])
	sorted_circles = sorted(distances, key=lambda elem: elem[0])

	return sorted_circles[0][1]

def save_video(frames):
	"""Function that saves the frames passed as parameter to a video file"""
	fourcc = cv.VideoWriter_fourcc(*'mp4v') 
	out = cv.VideoWriter('data/result/roger.avi', fourcc, 10, (frames[0].shape[1], frames[0].shape[0]))
	for i in range(len(frames) - 1):
		out.write(frames[i])
		cv.imshow('frame',frames[i])
		if cv.waitKey(1) & 0xFF == ord('q'):
			break
	cv.destroyAllWindows()
		
	out.release()

### Loading video data

In [3]:
def read_video(path, show=False, frame_by_frame=False):
	"""Function that read the video file in the path passed as parameter,
		and return each individual frame
	"""
	video = cv.VideoCapture(path)
	video_frames = []
	i = 0
	while(video.isOpened()):
		ret, frame = video.read()
		if ret:
			video_frames.append(frame)
			if show:
				if frame_by_frame:
					show_frame(frame, i)
				else:
					cv.imshow("Frame", frame)
					if cv.waitKey(25) & 0xFF == ord('q'):
						break
			i = i + 1
		else:
			break

	video.release()

	if cv.waitKey(0):
		cv.destroyAllWindows()

	return video_frames

### Ball detection

In [4]:
def find_ball_hough(frames, param=1, param0=1000, param1=30, param2=15, min_radius=5, max_radius=15):
	"""Function that estimates the position of the tennis ball inside each frame"""
	circle_frames = []
	circle_contour = []
	for i in range(len(frames)):
		frame = cv.cvtColor(frames[i], cv.COLOR_BGR2GRAY)
		frame = cv.GaussianBlur(frame, (7, 7), 0)

		circles = cv.HoughCircles(frame, cv.HOUGH_GRADIENT, param, param0, param1=param1, param2=param2, minRadius=min_radius, maxRadius=max_radius)
		if circles is not None:
			circles = np.uint16(np.around(circles))
			for i in circles[0,:]:
				# draw the outer circle
				cv.circle(frame,(i[0],i[1]),i[2],(0,255,0),2)
				# draw the center of the circle
				cv.circle(frame,(i[0],i[1]),2,(0,0,255),3)
		
			circle_frames.append(frame)
			circle_contour.append(circles[0])

	return circle_frames, circle_contour

### Kalman filter

In [5]:
def initialize_kalman(x_center, y_center, fps, covariance_coefficient, measurements_coefficient, skip_frames):
	"""Function that initializes all the required variables for the Kalman filter"""
	delta = fps * skip_frames

	# INITIALIZATION
	
	# defining the state vector
	s_pred = np.array([x_center, y_center, 0, 0, -9.8, 0])

	# defining state transition matrix, filling the matrix using the 
	# velocity, acceleration and position formulas
	# v = v_-1 + at
	# v^1 = v^2_0 + 2a(x - x_0)
	# x = x_-1 + v_0t + (at^2)/2

	Phi = np.eye(s_pred.shape[0])
	Phi[0][2] = delta
	Phi[0][4] = (delta**2)/2
	Phi[1][3] = delta
	Phi[1][5] = (delta**2)/2
	Phi[2][4] = delta
	Phi[3][5] = delta

	# defining the covariance matrix
	P_pred = np.eye(s_pred.shape[0])

	# defining the covariance noise matrix
	Q = covariance_coefficient*np.eye(s_pred.shape[0])

	# defining the measurements matrix
	H = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])

	# defining the measurements noise matrix
	R = measurements_coefficient*np.eye(2)

	return s_pred, Phi, P_pred, Q, H, R

In [21]:
def predict_kalman(Phi, P, Q, s):
	"""Implementation of the prediction step in the kalman filter algorithm"""
	# computing the projection
	s_pred = Phi @ s
	P_pred = ((Phi @ P) @ Phi.T) + Q

	return s_pred, P_pred

def update_kalman(x_center, y_center, H, P_pred, R, s_pred):
	"""Implementation of the update step in the Kalman filter algorithm"""
	m = np.array([x_center, y_center])

	# computing the gain
	T = H @ P_pred @ H.T + R
	K = P_pred @ H.T @ np.linalg.inv(T)

	# computing the innovation
	innovation = m - (H @ s_pred)
	s = s_pred + (K @ innovation)
	I = np.eye(len(s_pred))
	P = (I - K @ H) @ P_pred

	return s, P

def run_kalman(frames, fps, covariance_coefficient=1, measurement_coefficients=1, skip_frames=1):
	"""Function that applies the Kalman filter on the frames passed as parameter"""
	tracking = []
	tracked_frames = []
	print(measurement_coefficients)
	print(covariance_coefficient)
	# getting measurements
	circle_frames, ball_positions = find_ball_hough(frames, 1, frames[0].shape[0]*2, 30, 15, 20, 22)

	# initializing kalman
	initial_x_position, initial_y_position = set_initial_position(ball_positions)

	s_pred, Phi, P_pred, Q, H, R = initialize_kalman(initial_x_position, initial_y_position, fps, covariance_coefficient, measurement_coefficients, skip_frames)
	
	prev_measurement = [initial_x_position, initial_y_position]

	# iterating 
	for i in range(len(ball_positions)):

		# control if the measure is present
		if len(ball_positions[i]) > 0:
			# computing the nearest measure with respect to the previous one
			# center = compute_nearest_measure(i, ball_positions, prev_measurement)
			center = ball_positions[i]
			s, P = update_kalman(center[0][0], center[0][1], H, P_pred, R, s_pred)
		
		s_pred, P_pred = predict_kalman(Phi, P, Q, s)

		tracking.append(s_pred)
		prev_measurement = ball_positions[i][0]

	# marking the results on the image
	# for i in range(len(frames) - 1):
	# 	cv.circle(frames[i], (int(tracking[i][0]), int(tracking[i][1])), 20, (0, 255, 0), 4)
	# 	for j in range(len(ball_positions[i])):
	# 		cv.circle(frames[i], (ball_positions[i][j][0], ball_positions[i][j][1]), 10, (0, 0, 255), 4)
	# 	tracked_frames.append(frames[i])

	return tracked_frames, tracking

### Reading the video

In [26]:
image_path = "data/roger_forehand.avi"
frames = read_video(image_path)	

### Applying Kalman

In [27]:
tracked_frames, tracking = run_kalman(frames, 10, 1, 1000, 1)
path = []

# drawing the path
for i in range(len(frames)):
	for j in range(len(tracking)):
		cv.circle(frames[i], (int(tracking[j][0]), int(tracking[j][1])), 20, (0, 255, 0), 4)
		if j >= i:
			break
	path.append(frames[i])

run_frames(path)
# saving the video
save_video(path)

1000
1


In [20]:
frame, ball = find_ball_hough(frames, 1, frames[0].shape[0]*2, 30, 15, 20, 22)
run_frames(frame, one_by_one=True)

KeyboardInterrupt: 